## Setup

Please execute the cell(s) below to initialize the notebook environment.

In [ ]:
# @title Install dependencies
# !pip install poetry

In [ ]:
# @title Install SMorph Python module
# !pip install https://github.com/swanandlab/SMorph/releases/download/v0.1.1-alpha/SMorph-0.1.1.tar.gz

In [ ]:
# Imports
on_colab = 'google.colab' in str(get_ipython())

import warnings
warnings.filterwarnings('ignore')

if not on_colab:
  import napari
import smorph as sm
import smorph.util.autocrop as ac
import ipywidgets as widgets

from magicgui import magicgui
import winsound

In [ ]:
# Helper function
def view_3D(*args):
    """Views a 3D image."""
    n_images = len(args)
    if not on_colab and n_images > 0:
        with napari.gui_qt():
            viewer = napari.view_image(**args[0], ndisplay=3, scale=pipe.SCALE)
            for itr in range(1, n_images):
                if args[itr]['data'].max() > 1 and args[itr]['data'].max() % 1 == 0:
                    if 'colormap' in args[itr].keys():
                        del args[itr]['colormap']
                    if 'gamma' in args[itr].keys():
                        del args[itr]['gamma']
                    viewer.add_labels(**args[itr], scale=pipe.SCALE)
                else:
                    viewer.add_image(**args[itr], scale=pipe.SCALE)

---

---
## Step 0: Import Reference image

---

## Step 1: Import Microscopic Image of the Tissue

Set `TISSUE_IMAGE` to the path of the image file to be processed.

1. Deconvolution
2. Rolling ball background subtraction
3. CLAHE
4. ROI selection
5. Non-local means denoising using auto-calibrated parameters using J-Invariance

In [ ]:
TISSUE_IMAGE = 'E:/Garima Confocal/SAL,DMI, FLX ADN HALO_TREATMENT_28 DAYS/control_28 days/CONTROL_MSP2.1MB_4_LONG MARK_20X_SEC 1_LEFT HILUS_28 DAYczi.czi'  #@param
FILE_ROI = 'E:/Garima Confocal/SAL,DMI, FLX ADN HALO_TREATMENT_28 DAYS/control_28 days/CONTROL_MSP2.1MB_4_LONG MARK_20X_SEC 1_LEFT HILUS_28 DAYczi_ML.roi'
ROI_NAME = 'ML'

DECONV_ITR = 30
CLIP_LIMIT = .02

REF_IMAGE = 'E:/Garima Confocal/SAL,DMI, FLX ADN HALO_TREATMENT_28 DAYS/control_28 days/CONTROL_MSP3.1M_1_SINGLE MARK_20X_SEC 1_RIGHT HILUS_28 DAYczi.czi'
REF_FILE_ROI = 'E:/Garima Confocal/SAL,DMI, FLX ADN HALO_TREATMENT_28 DAYS/control_28 days/CONTROL_MSP3.1M_1_SINGLE MARK_20X_SEC 1_RIGHT HILUS_28 DAYczi_ML.roi'

cache_dir = 'Cache/'

pipe = ac.TissueImage(TISSUE_IMAGE, FILE_ROI, ROI_NAME, DECONV_ITR, CLIP_LIMIT, REF_IMAGE, REF_FILE_ROI, cache_dir)
winsound.Beep(440, 1000)

---

## Step 2: Select ROI using Polygonal Lasso Tool

Set two variables:
- `SELECT_ROI`: True, If you want to select ROI manually; else False
- `NAME_ROI`: Name of the manually selected ROI
- `FILE_ROI`: Path to the ROI file; else None

In [ ]:
view_3D({'data': pipe.imdeconvolved, 'colormap': 'magma', 'name': 'deconvolved'},
        {'data': pipe.impreprocessed, 'colormap': 'magma', 'name': 'preprocessed'},
        {'data': pipe.imdenoised, 'colormap': 'magma', 'name': 'denoised'})

---

## Step 3: Segmentation

### 3.1 Global threshold & color label cells

Set two parameters:
- `LOW_THRESH`: Pixel intensity value corresponding to faintest branch's edge
- `HIGH_THRESH`: Pixel intensity value corresponding to faintest soma

Understand their effect by configuring three parameters:
- `LOW_DELTA`: Pixel intensity value corresponding to change in `LOW_THRESH`
- `HIGH_DELTA`: Pixel intensity value corresponding to change in `HIGH_THRESH`
- `N_STEPS`: Number of steps of delta in threshold to take in both directions

In [ ]:
LOW_THRESH = .07  # .10 #.09
HIGH_THRESH = .12

LOW_DELTA = .01
HIGH_DELTA = .01
N_STEPS = 0
results = None
import skimage
%matplotlib inline
def test_thresholds(low_thresh_init, low_thresh, high_thresh_init,
                    high_thresh, low_delta, high_delta, n_steps):
  global results, LOW_THRESH, HIGH_THRESH, LOW_DELTA, HIGH_DELTA, N_STEPS
  LOW_THRESH, HIGH_THRESH, N_STEPS = low_thresh, high_thresh, n_steps
  if low_thresh_init is not None:
    LOW_THRESH = eval(f'skimage.filters.threshold_{low_thresh_init}(pipe.imdenoised[:, lly:ury, llx:urx])')
  if high_thresh_init is not None:
    if high_thresh_init == 'isodata':
      HIGH_THRESH = eval(f'skimage.filters.threshold_{high_thresh_init}(pipe.imdenoised[:, lly:ury, llx:urx])')
  LOW_DELTA, HIGH_DELTA = low_delta, high_delta
  results = ac.testThresholds(pipe.imdenoised, LOW_THRESH, HIGH_THRESH, LOW_DELTA,
                              HIGH_DELTA, N_STEPS, 'gist_earth')

if on_colab:
  _ = widgets.interact(test_thresholds,
                      low_thresh_init=[None, *sm.util.THRESHOLD_METHODS],
                      low_thresh=widgets.FloatSlider(LOW_THRESH, min=0, max=1, step=.01,
                                                      readout_format='.4f', layout=widgets.Layout(width='100%')),
                      high_thresh_init=[None, *sm.util.THRESHOLD_METHODS],
                      high_thresh=widgets.FloatSlider(HIGH_THRESH, min=0, max=1, step=.01,
                                                      readout_format='.4f', layout=widgets.Layout(width='100%')),
                      low_delta=widgets.FloatSlider(LOW_DELTA, min=0, max=1, step=.0005,
                                                    readout_format='.4f', layout=widgets.Layout(width='100%')),
                      high_delta=widgets.FloatSlider(HIGH_DELTA, min=0, max=1, step=.0005,
                                                      readout_format='.4f', layout=widgets.Layout(width='100%')),
                      n_steps=widgets.IntSlider(N_STEPS, min=0, max=10,
                                                layout=widgets.Layout(width='100%'))
  )
  # view_3D({'data': pipe.imdenoised, 'colormap': 'gray_r', 'name': 'denoised'}, *results)
else:
  viewer = napari.Viewer(ndisplay=3)
  viewer.add_image(pipe.impreprocessed, scale=pipe.SCALE)
  @magicgui(
    call_button="Test Thresholds",
    low_auto_thresh={'choices': [None, *sm.util.THRESHOLD_METHODS]},
    low_thresh={"widget_type": "FloatSlider", 'max': 1},
    high_auto_thresh={'choices': [None, *sm.util.THRESHOLD_METHODS]},
    high_thresh={"widget_type": "FloatSlider", 'max': 1},
    low_delta={"widget_type": "FloatSlider", 'max': 1},
    high_delta={"widget_type": "FloatSlider", 'max': 1}
  )
  def test_thresholds_gui(
      low_auto_thresh=None,
      low_thresh=LOW_THRESH,
      high_auto_thresh=None,
      high_thresh=HIGH_THRESH,
      low_delta=LOW_DELTA,
      high_delta=HIGH_DELTA,
      n_steps=N_STEPS
  ):
    global results, LOW_THRESH, HIGH_THRESH, LOW_DELTA, HIGH_DELTA, N_STEPS
    LOW_THRESH, HIGH_THRESH, N_STEPS = low_thresh, high_thresh, n_steps
    lly, llx, ury, urx = pipe.in_box
    if low_auto_thresh is not None:
      LOW_THRESH = eval(f'skimage.filters.threshold_{low_auto_thresh}(pipe.imdenoised[:, lly:ury, llx:urx])')
    if high_auto_thresh is not None:
      # if high_auto_init == 'isodata':
      HIGH_THRESH = eval(f'skimage.filters.threshold_{high_auto_thresh}(pipe.imdenoised[:, lly:ury, llx:urx])')
    LOW_DELTA, HIGH_DELTA = low_delta, high_delta
    results = ac.core._testThresholds(pipe.imdenoised, LOW_THRESH, HIGH_THRESH, LOW_DELTA,
                                      HIGH_DELTA, N_STEPS)
    past_state = viewer.window.qt_viewer.view.camera.get_state()
    viewer.layers.clear()

    viewer.add_image(pipe.impreprocessed, scale=pipe.SCALE)

    n_images = len(results)
    for itr in range(n_images):
      viewer.add_labels(**results[itr], scale=pipe.SCALE)
    viewer.window.qt_viewer.view.camera.set_state(past_state)

  viewer.window.add_dock_widget(test_thresholds_gui)
  test_thresholds_gui()

In [ ]:
print(LOW_THRESH, HIGH_THRESH)

pipe.segment(LOW_THRESH, HIGH_THRESH)

In [ ]:
view_3D({'data': pipe.imsegmented, 'colormap': 'inferno', 'name': 'segmented'},
        #{'data': pipe.filtered_labels, 'colormap': 'gray', 'gamma': .8, 'name': 'filtered_labels'},
        {'data': pipe.labels, 'colormap': 'gist_earth', 'gamma': .8, 'name': 'labels'})

### 3.3 Visualize segmented cells to determine cutoff volumes

#### 3.3.1 Check segmented cells on whole image

In [ ]:
pipe.volume_cutoff()

In [ ]:
somas_load_path = 'Autocropped/CONTROL_MSP2.1MB_4_LONG MARK_20X_SEC 1_LEFT HILUS_28 DAYczi-ML/.somas_estimates.npy'
pipe.approximate_somas(src=somas_load_path)

# napari clump sep

In [ ]:
seed_src = 'Autocropped/CONTROL_MSP2.1MB_4_LONG MARK_20X_SEC 1_LEFT HILUS_28 DAYczi-ML/.somas_estimates.npy'
pipe.separate_clumps(seed_src)

In [ ]:
len(pipe.regions)

#### 3.3.2: Check batches of objects

In [ ]:
pipe.show_segmented('grid', 50)

#### 3.3.2: Check individual objects
Select individual objects using `OBJ_INDEX`.

In [ ]:
pipe.show_segmented()

#### 3.3.3: Refine soma approximations
Select individual objects using `OBJ_INDEX`.

In [ ]:
pipe.refine_soma_approx()

In [ ]:
pipe.separate_clumps()

---

## Step 4: Export autocropped 3D cells or 2D max intensity projections

Set two parameters:
- `LOW_VOLUME_CUTOFF`: to filter out noise/artifacts
- `HIGH_VOLUME_CUTOFF`: to filter out cell clusters

For choosing between 3D segmented cells or 2D max intensity projections:
- Set `OUTPUT_OPTION` = '3d' for 3D cells, or
- Set `OUTPUT_OPTION` = 'mip' for Max Intensity Projections.

In [ ]:
pipe.export_cropped()